<p align="center">
<b><font size="9" color="#000080">PORTICO PLANO ETABS</font></b>
<div align="center">
    <div style="display: inline-block;">
    <span style="font-size: 18px;position: relative; bottom: 13px;">Richard Cueto Calderon | Update :7/05/204 | API ETBAS | Python</span>
    </div>
</div>
</p>

Vamos a crear un portico de dos paños

Input:
+ L1
+ L2
+ H1
+ Fc
+ Seccion de columnas y vigas

In [1]:
import os
import sys
import comtypes.client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Etabas

In [2]:
AttachToInstance = False
SpecifyPath = True
ProgramPath = "C:\Program Files\Computers and Structures\ETABS 19\ETABS.exe"
APIPath = r'D:\Users\User(D)\Documentos\GitHub\freeproyect\Etabs\Modelos'
if not os.path.exists(APIPath):
    try:
        os.makedirs(APIPath)
    except OSError:
        pass
ModelPath = APIPath + os.sep + 'Portico.edb'
helper = comtypes.client.CreateObject('ETABSv1.Helper')
helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)
if AttachToInstance:
    try:
        myETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject") 
    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)
else:
    if SpecifyPath:
        try:
            myETABSObject = helper.CreateObject(ProgramPath)
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program from " + ProgramPath)
            sys.exit(-1)
    else:
        try: 
            myETABSObject = helper.CreateObjectProgID("CSI.ETABS.API.ETABSObject") 
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program.")
            sys.exit(-1)
    myETABSObject.ApplicationStart()
SapModel = myETABSObject.SapModel
SapModel.InitializeNewModel()

0

## Modelo

In [3]:
# Creamos un nuevo modelo en blanco
ret=SapModel.File.NewBlank()

In [4]:
# Cambiamos unidades a ton_m
Tonf_m_C=12
ret = SapModel.SetPresentUnits(Tonf_m_C)

In [5]:
# Definicion de material concreto
fc=280
material_type=2
material_name="fc"+str(fc)+"kg/cm2" #fc=280kg/cm2
ret=SapModel.PropMaterial.SetMaterial(material_name, material_type)
# Definir propiedades mecanicas del material
E=15000*fc**0.5             #kg/cm2
v=0.2
CT=0.0000055
ret=SapModel.PropMaterial.SetMPIsotropic(material_name,E,v,CT)

In [6]:
# Creamos seccion de viga y columna
# Viga
b_v=0.3
h_v=0.6
nombre_viga='v '+str(b_v)+'x'+str(h_v)          #V 0.30x0.60
ret = SapModel.PropFrame.SetRectangle(nombre_viga, material_name, h_v, b_v)

# Columna
b_c=0.45
h_c=0.45
nombre_columna='v '+str(b_c)+'x'+str(h_c)          #V 0.45x0.45
ret = SapModel.PropFrame.SetRectangle(nombre_columna, material_name, h_c, b_c)

In [ ]:
# definimos modificaciones
ModValue = [1000, 0, 0, 1, 1, 1, 1, 1]
ret = SapModel.PropFrame.SetModifiers(nombre_viga, ModValue)
ret = SapModel.PropFrame.SetModifiers(nombre_columna, ModValue)

**CREACION DE PORTICO 3D**

Esta creacion ser de manera manual y mecanica

+ Creacion de Frame por coordendas
+ Ingresamos paramteros


In [ ]:
h=3
L=4
paños=10
niveles=5

for j in range(niveles):
    #columnas
    for i in range(paños+1):
        FrameName = ' '
        [FrameName, ret] = SapModel.FrameObj.AddByCoord(i*L, 0, j*h,i*L, 0, (j+1)*h, FrameName, nombre_columna, '', 'Global')

    # vigas
    for i in range(paños):
        FrameName = ' '
        [FrameName, ret] = SapModel.FrameObj.AddByCoord(i*L, 0, (j+1)*h, (i+1)*L, 0, (j+1)*h, FrameName, nombre_viga, '', 'Global')


In [ ]:
# Agregamos muro
for j in range(niveles):
    nameWall=""
    x=[0,L,L,0]
    y=[0,0,0,0]
    z=[0,0,(j+1)*h,(j+1)*h]
    ret = SapModel.AreaObj.AddByCoord(4, x, y, z, nameWall)

    nameWall=""
    x=[(paños-1)*L,paños*L,paños*L,(paños-1)*L]
    y=[0,0,0,0]
    z=[0,0,(j+1)*h,(j+1)*h]
    ret = SapModel.AreaObj.AddByCoord(4, x, y, z, nameWall)

In [ ]:
#assign point object restraint at base
Restraint = [True, True, True, True, True, True]
for i in range(paños+1):
    Pointi,Pointj= ' ',' '
    [Pointi, Pointj, ret] = SapModel.FrameObj.GetPoints(str(i+1), Pointi, Pointj)
    ret = SapModel.PointObj.SetRestraint(Pointi, Restraint)

In [ ]:
# Creamos un LoadPattern
ret = SapModel.LoadPatterns.Add('FH', 8, 0, True)
for j in range(niveles):
    for i in range(paños):
        # Asignar carga distribuida
        ret = SapModel.FrameObj.SetLoadDistributed(str(paños+2+i+(2*paños+1)*j),'Dead',1,2,0,1,-0.4,-0.4,'Local')
        ret = SapModel.FrameObj.SetLoadDistributed(str(paños+2+i+(2*paños+1)*j),'Live',1,2,0,1,-0.25,-0.25,'Local')

In [ ]:

# Asignar carga puntual
PointLoadValue = [1,0,0,0,0,0]
Pointi=""
Pointj=""
[Pointi, Pointj, ret] = SapModel.FrameObj.GetPoints("1", Pointi, Pointj)
ret = SapModel.PointObj.SetLoadForce(Pointj,'FH',PointLoadValue)

In [ ]:
#refresh view, update (initialize) zoom
ret = SapModel.View.RefreshView(0, False)

In [ ]:
#save model
ret = SapModel.File.Save(ModelPath)

In [ ]:
#run model (this will create the analysis model)
ret = SapModel.Analyze.RunAnalysis()

# EXTRAER DATOS DEL ETABS

El codigo proporcionado nos permitira extraer cualquier tabla que se encuentre en el programa etabs.

El trabajo de esta sera identificar las tablas con las que queremos trabajar.

In [4]:
def conect_to_etabs():
    helper = comtypes.client.CreateObject('ETABSv1.Helper')
    helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)
    try:
        myETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject")
        print('Conected to ETABS model')
    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)
    SapModel = myETABSObject.SapModel
    return SapModel,myETABSObject,helper

SapModel,myETABSObject,helper=conect_to_etabs()

Conected to ETABS model


In [ ]:
name_table="Base Reactions"
name_table="Element Forces - Beams"
table=SapModel.DatabaseTables.GetTableForDisplayArray (name_table,GroupName="")
colums=table[2]
noOfRows=table[3]
val=np.array_split(table[4],noOfRows)
df=pd.DataFrame(val)
df.columns=colums
df

In [ ]:
df

In [ ]:
# df=df.drop(columns='FX')
# df=df.drop(columns='FY')
# df=df.drop(columns='MZ')
# df=df.drop(columns='X')
# df=df.drop(columns='Y')
# df=df.drop(columns='Z')

In [ ]:
df